In [10]:
pip install geopy

     -------------------------------------- 125.4/125.4 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time
import googlemaps
import pandas as pd
import csv
from pprint import pprint
import openpyxl
from geopy.geocoders import GoogleV3

In [2]:
#defining API key
API_KEY = open('API.txt','r').read()
map_client = googlemaps.Client(API_KEY)

Testing code to reverse search for Neighbourhood names

In [24]:
# Initializing the Google Maps client
gmaps = googlemaps.Client(API_KEY)

# Defining the coordinates
latitude = 39.22830342820063
longitude = -76.60081926609222

# Performing reverse geocoding to get location details
reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

# Extracting neighborhood information if available
neighborhood = None
for result in reverse_geocode_result:
    for component in result['address_components']:
        if 'neighborhood' in component['types']:
            neighborhood = component['long_name']

if neighborhood:
    print(f'Neighborhood: {neighborhood}')
else:
    print('Neighborhood information not found.')

Neighborhood: Brooklyn Homes


In [5]:
Balt_crime=pd.read_csv('Crime_Data_7_22_2023.csv')

C:\Users\stean\AppData\Local\Temp\ipykernel_23668\839219139.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  Balt_crime=pd.read_csv('Crime_Data_7_22_2023.csv')


Getting unique names of neighborhoods

In [6]:
Balt_unique = Balt_crime.drop_duplicates(subset=['Neighborhood'])

Balt_unique.to_csv('unique_neighbourhood2.csv', index=False)

Balt_unique = Balt_unique.dropna(subset=['Neighborhood'])

Balt_unique.to_csv('unique_neighbourhood2.csv', index=False)

Now that i have the unique neighborhood names i will use that with the google maps to reverse call their actual names and replace them in the dataset 

In [17]:
Balt_unique = pd.read_csv('unique_neighbourhood2.csv')

# Converting the "Latitude" column to strings and then clean it
Balt_unique['Latitude'] = Balt_unique['Latitude'].astype(str).str.replace("'", '')

Saving to CSV

In [18]:
Balt_unique.to_csv('unique_neighbourhood3.csv', index=False)

Formatting coordinates to show the right way 

In [22]:
Balt_nnames = pd.read_csv('unique_neighbourhood2.csv')

# Concatenating "Latitude" and "Longitude" columns with a comma in between
Balt_nnames['Coordinates'] = Balt_nnames['Latitude'].astype(str) + ',' + Balt_nnames['Longitude'].astype(str)

Balt_nnames.to_csv('unique_neighbourhoods.csv', index=False)

Trying out code to get the names of the coordinates I formatted 

In [19]:
#Loading DataFrame with latitude and longitude columns
Balt_nnames = pd.read_csv('unique_neighbourhood2.csv')

#Creating a Google geocoder object
geolocator = GoogleV3(API_KEY)

#creating function to get neighborhood names
def get_neighborhood_name(row):
    try:
        location = geolocator.reverse((row['Latitude'], row['Longitude']), exactly_one=True)
        if location and 'neighborhood' in location.raw['address_components'][0]['types']:
            return location.raw['address_components'][0]['long_name']
        else:
            return 'Not Found'
    except:
        return 'Error'

#Applying function to DataFrame to create a new column
Balt_nnames['neighborhood_name'] = Balt_nnames.apply(get_neighborhood_name, axis=1)

#Saving DataFrame with the new column
Balt_nnames.to_csv('neighborhood_data_with_names.csv', index=False)

Above code did not work so i had to get a little more specific

In [25]:
#initializing the Google Maps client
gmaps = googlemaps.Client(API_KEY)

#Loading DataFrame with crime records, including "Latitude" and "Longitude" columns
crime_df = pd.read_csv('unique_neighbourhoods.csv')

#Creating function to get neighborhood names
def get_neighborhood_name(row):
    latitude = row['Latitude']
    longitude = row['Longitude']

    #Performing reverse geocoding to get location details
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

    #Extracting neighborhood information if available
    neighborhood = None
    for result in reverse_geocode_result:
        for component in result['address_components']:
            if 'neighborhood' in component['types']:
                neighborhood = component['long_name']

    return neighborhood if neighborhood else 'Not Found'

#Applying function to DataFrame to create a new column
crime_df['neighborhood_names'] = crime_df.apply(get_neighborhood_name, axis=1)

# Saving DataFrame with the new "neighborhood_names" column
crime_df.to_csv('crime_records_with_neighborhoodss.csv', index=False)


Putting the coordinates of the neighborhoods i have now by the their respective neighborhood names by merging

In [26]:

balt_crime = pd.read_csv('Crime_Data_7_22_2023.csv')

# Merging the DataFrames based on the "Neighborhood" column
balt_crime = balt_crime.merge(crime_df[['Neighborhood', 'neighborhood_names']],
                              left_on='Neighborhood',
                              right_on='Neighborhood',
                              how='left')


# Saving the merged DataFrame
balt_crime.to_csv('balt_crime_with_neighborhood_names.csv', index=False)


C:\Users\stean\AppData\Local\Temp\ipykernel_23668\638121339.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crime = pd.read_csv('Crime_Data_7_22_2023.csv')


updated crime dataset

In [27]:
balt_crime_update = pd.read_csv('Part_1_Crime_Data_update.csv')

# Merge the DataFrames based on the "Neighborhood" column
balt_crime_update = balt_crime_update.merge(crime_df[['Neighborhood', 'neighborhood_names']],
                              left_on='Neighborhood',
                              right_on='Neighborhood',
                              how='left')


#Saving the merged DataFrame
balt_crime_update.to_csv('balt_crime_updated.csv', index=False)

C:\Users\stean\AppData\Local\Temp\ipykernel_23668\3219203420.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crime_update = pd.read_csv('Part_1_Crime_Data_update.csv')


now i will use the address from the property dataset to get the unique neigbourhood names for the property dataset and remove all the neighbour hoods that are not present in that for the crime dataset

In [28]:
prop = pd.read_csv('addresses_almost_cleaned.csv')

# Get a list of unique neighborhoods
unique_neighborhoods = prop['neighbourhood'].unique()

# Convert the array of unique neighborhoods to a list
unique_neighborhoods_list = unique_neighborhoods.tolist()

# Print or use the unique neighborhoods as needed
print(unique_neighborhoods_list)


['Mount Washington', 'Fells Point', 'Clifton', 'Downtown', 'Arlington', 'Carroll', 'Purnell', 'Greater Northwood Covenant Association', 'Northwood', 'Locust Point', nan, 'Lakeland', 'Bolton Hill', 'Highlandtown', 'Gwynn Oak', 'Otterbein', 'Franklintown', 'Cherry Hill', 'Woodberry', 'Tuscany - Canterbury', 'East Case', 'New North Roland Park - Poplar Hill', 'Riverside', 'Druid Hill Park', 'Federal Hill-Montgomery', 'Garwyn Oaks', 'West Forest Park', 'Inner Harbor', "Ridgely's Delight", 'Ednor Gardens - Lakeside', 'Raspeburg', 'Charles Village', 'Medfield', 'Sharp Leadenhall', 'Harford - Echodale - Perring Parkway', 'Mount Holly', 'Govans', 'Station North', 'Upper Fells Point', 'Hampden', 'Old Goucher', 'Brooklyn', 'Pen Lucy', 'Oakenshawe', 'Harwood', 'Coldstream - Homestead - Montebello', 'Bridgeview-Greenlawn', 'Gay Street', 'Franklin', 'South Baltimore', 'Middle East', 'Pigtown', 'Orangeville', 'Better Waverly', 'Butchers Hill', 'Evergreen Lawn', 'Morrell Park', 'Glenham-Belford', 'We

Filtering the DataFrame to keep only the rows with matching neighborhood names

In [29]:
filtered_bcu = balt_crime_update[balt_crime_update['neighborhood_names'].isin(unique_neighborhoods_list)]

#Saving the filtered DataFrame
filtered_bcu.to_csv('filtered_balt_crime.csv', index=False)

Removing rows where "neighborhood_names" is empty

In [30]:
filtered_bcu = filtered_bcu[filtered_bcu['neighborhood_names'].notna()]

#Saving the filtered DataFrame
filtered_bcu.to_csv('filtered_balt_crime.csv', index=False)

In [31]:
null_values = filtered_bcu.isnull().sum()
print(null_values)

X                          0
Y                          0
RowID                      0
CCNumber                   0
CrimeDateTime              0
CrimeCode                  0
Description                0
Inside_Outside         44714
Weapon                386608
Post                       6
Gender                 83368
Age                   100958
Race                   17716
Ethnicity             458234
Location                1163
Old_District           16250
New_District          497906
Neighborhood               0
Latitude                   0
Longitude                  0
GeoLocation                0
PremiseType            45011
Total_Incidents            0
neighborhood_names         0
dtype: int64


Removing columns not needed

In [32]:
#List of columns to keep
columns_to_keep = ['Weapon', 'Inside_Outside', 'Description', 'GeoLocation', 'Latitude', 'Longitude']

#Filtering the DataFrame to keep only the specified columns
filtered_bcu = filtered_bcu[columns_to_keep]

#Saving the filtered DataFrame
filtered_bcu.to_csv('final_baltcrime.csv', index=False)


Getting the google coordinates of the neighborhood names

In [ ]:
crime_df = pd.read_csv('crime_records_with_neighborhoodss.csv')

# Creating a Google geocoder object
geolocator = GoogleV3(API_KEY)  # Replace 'YOUR_API_KEY' with your Google API key

#Creating function to get coordinates for neighborhood names within Baltimore, Maryland, USA
def get_coordinates(row):
    neighborhood_name = row['neighborhood_names']
    address = f'{neighborhood_name}, Baltimore, Maryland, USA'
    location = geolocator.geocode(address)
    if location:
        return f'{location.latitude}, {location.longitude}'
    else:
        return 'Not Found'

#Applying function to DataFrame to create a new column
crime_df['google_coordinates'] = crime_df.apply(get_coordinates, axis=1)

In [8]:
#Saving the DataFrame with the new "google_coordinates" column
crime_df.to_csv('aaaupdated_dataset.csv', index=False)

Trying code to populate the neighborhood_names column based on the common Neighborhood column

In [ ]:
balt_crime_update = pd.read_csv('filtered_balt_crime.csv')
crime_df = pd.read_csv('aaaupdated_dataset.csv')

#Merging the DataFrames based on the "Neighborhood" column
balt_crime_update = balt_crime_update.merge(crime_df[['Neighborhood', 'neighborhood_names']],
                              left_on='Neighborhood',
                              right_on='Neighborhood',
                              how='left')


# Save the merged DataFrame
balt_crime_update.to_csv('aaabalt_crime_updated.csv', index=False)

In [12]:
balt_crime = pd.read_csv('filtered_balt_crime.csv')
crime_df = pd.read_csv('aaaupdated_dataset.csv')

#Merging the DataFrames based on the "neighborhood_names" column
balt_crime = balt_crime.merge(crime_df[['neighborhood_names', 'google_coordinates']],
                              on='neighborhood_names',
                              how='left')

#Saving the updated `balt_crime` DataFrame
balt_crime.to_csv('aaabalt_crime_updated.csv', index=False)

C:\Users\stean\AppData\Local\Temp\ipykernel_4100\2251626245.py:2: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crime = pd.read_csv('filtered_balt_crime.csv')


In [10]:
#Loading balt_crime DataFrame and crime_df DataFrame
balt_crime = pd.read_csv('filtered_balt_crime.csv')
crime_df = pd.read_csv('aaaupdated_dataset.csv')

#Merging the DataFrames based on the "neighborhood_names" column
balt_crime = balt_crime.merge(crime_df[['neighborhood_names', 'google_coordinates']],
                              left_on='neighborhood_names',
                              right_on='neighborhood_names',
                              how='left')

#Saving the merged DataFrame
balt_crime.to_csv('aaabalt_crime_updated.csv', index=False)


C:\Users\stean\AppData\Local\Temp\ipykernel_4100\1849811950.py:2: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crime = pd.read_csv('filtered_balt_crime.csv')


retrying my the work

In [13]:
# Load your DataFrame
prop2 = pd.read_csv('filtered_balt_crime.csv')

# Get a list of unique neighborhoods
unique_neighborhoods2 = prop2['neighborhood_names'].unique()

# Convert the array of unique neighborhoods to a list
unique_neighborhoods_list2 = unique_neighborhoods2.tolist()

# Print or use the unique neighborhoods as needed
print(unique_neighborhoods_list2)

C:\Users\stean\AppData\Local\Temp\ipykernel_4100\2719237036.py:2: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  prop2 = pd.read_csv('filtered_balt_crime.csv')


['Downtown', 'Druid Hill Park', 'Northwood', 'Harford - Echodale - Perring Parkway', 'Arlington', 'Fells Point', 'Highlandtown', 'East Baltimore Midway', 'Upper Fells Point', 'Franklin', 'Clifton', 'Mount Holly', 'Middle East', 'Morrell Park', 'Medfield', 'Raspeburg', 'Govans', 'Carroll', 'Garwyn Oaks', 'East Case', 'Barclay', 'Curtis Bay', 'Coldstream - Homestead - Montebello', 'Walbrook', 'Barre Circle', 'Pigtown', 'Elwood Park', 'Greater Northwood Covenant Association', 'Rosemont', 'Cherry Hill', 'Old Goucher', 'Abell', 'Gwynn Oak', 'Better Waverly', 'Lauraville', 'Butchers Hill', 'Coppin Heights', 'Charles Village', 'Gay Street', 'Lakeland', 'Woodberry', "Ridgely's Delight", 'Madison - Eastend', 'Hollins Market', 'Riverside', 'Mosher', 'West Forest Park', 'New North Roland Park - Poplar Hill', 'Federal Hill-Montgomery', 'Hampden', 'Mount Washington', 'Forest Park', 'Windsor Hills', 'Wyndhurst', 'Western', 'Harwood', 'Evergreen Lawn', 'South Baltimore', 'Brooklyn', 'Remington', 'Wym

In [15]:
balt_crime = pd.read_csv('filtered_balt_crime.csv')

# Filter the DataFrame to keep only the rows with matching neighborhood names
filtered_bcu2 = balt_crime[balt_crime['neighborhood_names'].isin(unique_neighborhoods_list2)]

# Save the filtered DataFrame
filtered_bcu2.to_csv('filtered_balt_crime2.csv', index=False)

# Remove rows where "neighborhood_names" is empty
filtered_bcu2 = filtered_bcu2[filtered_bcu2['neighborhood_names'].notna()]

# Save the filtered DataFrame
filtered_bcu2.to_csv('filtered_balt_crime2.csv', index=False)

C:\Users\stean\AppData\Local\Temp\ipykernel_4100\2095146642.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crime = pd.read_csv('filtered_balt_crime.csv')


In [16]:
#Droping duplicate rows based on the neighborhood_names column 
balt_crime = balt_crime.drop_duplicates(subset="neighborhood_names")

#Saving the dataframe to a new csv file 
balt_crime.to_csv('filtered_balt_crime2.csv', index=False)

In [17]:
#Droping duplicate rows based on the neighborhood_names column 
balt_crime = balt_crime.drop_duplicates(subset="Neighborhood")

#Saving the dataframe to a new csv file 
balt_crime.to_csv('filtered_balt_crime3.csv', index=False)

new getting unique neighbourhood of new dataset

In [18]:
#getting unique neighbourhood

Balt_crime=pd.read_csv('filtered_balt_crime.csv')

Balt_unique = Balt_crime.drop_duplicates(subset=['Neighborhood'])

Balt_unique.to_csv('unique_neighbourhoodc.csv', index=False)

Balt_unique = Balt_unique.dropna(subset=['Neighborhood'])

Balt_unique.to_csv('unique_neighbourhoodc.csv', index=False)

C:\Users\stean\AppData\Local\Temp\ipykernel_4100\259158407.py:3: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Balt_crime=pd.read_csv('filtered_balt_crime.csv')


Reverse search for neighborhood names

In [20]:
#Loading dataset with the "neighborhood_names" column
crime_dfb = pd.read_csv('unique_neighbourhoodc.csv')

#Creating a Google geocoder object
geolocator = GoogleV3(API_KEY)

#creating Function to get coordinates for neighborhood names within Baltimore, Maryland, USA
def get_coordinates(row):
    neighborhood_name = row['neighborhood_names']
    address = f'{neighborhood_name}, Baltimore, Maryland, USA'
    location = geolocator.geocode(address)
    if location:
        return f'{location.latitude}, {location.longitude}'
    else:
        return 'Not Found'

#Applying the function to your DataFrame to create a new column
crime_dfb['google_coordinates'] = crime_dfb.apply(get_coordinates, axis=1)

In [21]:
#Saving the DataFrame with the new "google_coordinates" column
crime_dfb.to_csv('aacupdated_dataset.csv', index=False)

In [22]:
#Loading balt_crime DataFrame and crime_df DataFrame
balt_crim = pd.read_csv('filtered_balt_crime.csv') #new crime dataset
crime_d = pd.read_csv('aacupdated_dataset.csv') #unique crime dataset with google coordinates

# Merge the DataFrames based on the "neighborhood_names" column
balt_crim = balt_crim.merge(crime_d[['neighborhood_names', 'google_coordinates']],
                              left_on='neighborhood_names',
                              right_on='neighborhood_names',
                              how='left')


# Save the merged DataFrame
balt_crim.to_csv('aaabalt_crime_updated.csv', index=False)

C:\Users\stean\AppData\Local\Temp\ipykernel_4100\378251046.py:2: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  balt_crim = pd.read_csv('filtered_balt_crime.csv')


Merging was still giving me duplicate rows even after starting over, I shared with Sean to assist